# Converting XML data to pandas dataframe and Saving the dataframe as PostHistory.csv in processed data folder

In [59]:
import pandas as pd
import numpy as np

In [60]:
post_df=pd.read_csv("/home/kick7/Desktop/projects/sml/fp/sof_user_churn/data/processed/posts.csv")

post_df.drop(columns=['LastEditorUserId','LastEditDate','LastActivityDate'],inplace=True)

post_df.dropna(inplace=True)

post_df[['OwnerUserId']]=post_df[['OwnerUserId']].astype(int)


(25488, 8)
(25488, 5)
(25410, 5)


,OwnerUserId,PostTypeId,CreationDate,Score,CommentCount
0,5,1,2014-05-13T23:58:30.457,9,1
1,36,1,2014-05-14T00:11:06.457,4,4
2,51,2,2014-05-14T00:36:31.077,5,0
3,22,2,2014-05-14T00:53:43.273,12,1
4,66,1,2014-05-14T01:25:59.677,21,1


# Droping complete row if NA present

In [61]:
post_df['CreationDate']=pd.to_datetime(post_df['CreationDate'])
post_df['diff'] = post_df.sort_values(['OwnerUserId','CreationDate'], ascending=[True, True]).groupby('OwnerUserId')['CreationDate'].diff()
post_df=post_df.sort_values(['OwnerUserId', 'CreationDate'], ascending=[True, True]).reset_index(drop=True)
post_df=post_df[post_df['OwnerUserId']!=-1].reset_index(drop=True)


(25410, 5)
(25319, 6)


In [62]:
post_df=post_df.fillna(0)
print(post_df.head())

   OwnerUserId  PostTypeId            CreationDate  Score  CommentCount  \
0            5           1 2014-05-13 23:58:30.457      9             1   
1            9           2 2014-05-14 10:41:23.823     77             4   
2            9           2 2014-05-15 15:06:24.600      5             0   
3           11           2 2014-06-10 07:42:57.470      5             0   
4           11           2 2014-10-03 16:07:30.327      4             0   

                      diff  
0          0 days 00:00:00  
1          0 days 00:00:00  
2   1 days 04:25:00.777000  
3          0 days 00:00:00  
4 115 days 08:24:32.857000  


In [79]:
inp=3
k=inp+1
print(post_df.shape)
k_df=post_df.groupby('OwnerUserId').filter(lambda x:x['CreationDate'].count()>=k)
k_df=k_df.sort_values(["OwnerUserId","CreationDate"]).reset_index()
k_df['diff']=pd.to_timedelta(k_df['diff'])
print(type(k_df['diff'][0]))
print(k_df.shape)
#k_df['diff']=k_df['diff'].dt.minute
k_df.head()

(25319, 6)
<class 'pandas._libs.tslibs.timedeltas.Timedelta'>
(14960, 7)


,index,OwnerUserId,PostTypeId,CreationDate,Score,CommentCount,diff
0,6,14,2,2014-05-14 05:44:29.340,29,0,0 days 00:00:00
1,7,14,2,2014-05-14 06:26:27.163,89,9,0 days 00:41:57.823000
2,8,14,2,2014-05-14 22:43:23.587,4,0,0 days 16:16:56.424000
3,9,14,2,2014-06-10 21:50:51.347,29,0,26 days 23:07:27.760000
4,10,14,2,2016-10-14 01:18:50.043,2,0,856 days 03:27:58.696000


In [94]:
days=365/2
l_k_df=k_df.sort_values(['OwnerUserId','CreationDate'], ascending=[True, True]).groupby('OwnerUserId').nth(k-1).reset_index()
l_k_df.drop(columns=['index'],inplace=True)
print(l_k_df.head())
c_k_df=l_k_df.sort_values(['OwnerUserId','CreationDate'], ascending=[True, True]).groupby('OwnerUserId').filter(lambda x:x['diff']>=pd.to_timedelta(str(days)+' days'))
nc_k_df=l_k_df.sort_values(['OwnerUserId','CreationDate'], ascending=[True, True]).groupby('OwnerUserId').filter(lambda x:x['diff']<pd.to_timedelta(str(days)+' days'))
l_oc_k_df=post_df.groupby('OwnerUserId').filter(lambda x:x['CreationDate'].count()==k-1)
oc_k_df=l_oc_k_df.sort_values(['OwnerUserId','CreationDate'], ascending=[True, True]).groupby('OwnerUserId').nth(k-2).reset_index()


print(k_df.shape)
print(c_k_df.shape)
print(oc_k_df.shape)
print(nc_k_df.shape)

Churn_users= c_k_df['OwnerUserId'].tolist()
Other_churn= oc_k_df['OwnerUserId'].tolist()
NonChurn_users=nc_k_df['OwnerUserId'].tolist()
Churn_users.extend(Other_churn)
print(len(set(Churn_users)),len(set(NonChurn_users)),len(set(Other_churn)))

   OwnerUserId  CommentCount            CreationDate  PostTypeId  Score  \
0           14             0 2014-06-10 21:50:51.347           2     29   
1           21             3 2014-05-18 21:25:29.040           2      7   
2           26             0 2014-05-15 07:47:44.710           2      8   
3           53             0 2014-05-16 15:35:51.420           5      0   
4           59             7 2014-05-27 12:07:45.920           2      8   

                     diff  
0 26 days 23:07:27.760000  
1  3 days 07:55:24.770000  
2  0 days 13:19:48.667000  
3  1 days 12:16:11.060000  
4  8 days 13:51:26.620000  
(14960, 7)
(81, 6)
(546, 6)
(1085, 6)
627 1085 546
